In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import cv2 as cv
import matplotlib.pyplot as plt

X_list = []
labels = []
countA = 0;
countB = 0;
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if dirname == '/kaggle/input/food11-image-dataset/training/Fried food':
            if countA >= 25:
                continue
            X_list.append(
                cv.cvtColor(
                    cv.imread(os.path.join(dirname, filename))
                    ,cv.COLOR_BGR2RGB)
            )
            labels.append('FriedFood')
            countA = countA + 1
        elif dirname == '/kaggle/input/food11-image-dataset/training/Meat':
            if countB >= 25:
                continue
            X_list.append(cv.cvtColor(
                    cv.imread(os.path.join(dirname, filename))
                    ,cv.COLOR_BGR2RGB)
            )
            labels.append('Meat')
            countB = countB+1
        if countA >= 25 and countB >= 25:
            break


X = np.asarray(X_list)

# print(image)
# print(image.shape)
# plt.imshow(image_cvt)

In [3]:
def meanMoment(channel):
    sumValue = 0
    countValue = 0
    for i in range(len(channel)):
        for j in range(len(channel[i])):
            #if(channel[i][j] < 99):
            if(channel[i][j] < 200):
                sumValue += channel[i][j]
                countValue += 1
    if(countValue == 0):
        return 0
    else:
        return sumValue/countValue

def varianceMoment(channel, meanChannel):
    sumValue = 0
    countValue = 0
    for i in range(len(channel)):
        for j in range(len(channel[i])):
            #if(channel[i][j] < 99):
            if(channel[i][j] < 200):
                sumValue += np.power(channel[i][j] - meanChannel,2)
                countValue += 1
    if(countValue == 0):
        return 0
    else:
        return np.sqrt(sumValue/countValue)

def skewnessMoment(channel, meanChannel):
    sumValue = np.int64(0)
    countValue = 0
    for i in range(len(channel)):
        for j in range(len(channel[i])):
            #if(channel[i][j] < 99):
            if(channel[i][j] < 200):
                sumValue += np.power(channel[i][j] - meanChannel,3)
                countValue += 1
    if(countValue == 0):
        return 0
    else:
        return np.cbrt(sumValue/countValue)
    
def getColorMoment(channel):
    meanChannel = meanMoment(channel)
    varChannel = varianceMoment(channel, meanChannel)
    skewChannel = skewnessMoment(channel, meanChannel)
    #return meanChannel, varChannel, skewChannel
    return meanChannel, varChannel, skewChannel

In [5]:
df = pd.DataFrame()

meanR_list = []
varR_list = []
skewR_list = []
meanG_list = []
varG_list = []
skewG_list = []
meanB_list = []
varB_list = []
skewB_list = []
aRatio_list = []
roundness_list = []

for i in range(len(X)):
    h = X[i].shape[0]
    w = X[i].shape[1]
    ymin, ymax, xmin, xmax = h//3, h*2//3, w//3, w*2//3
    crop = X[i][ymin:ymax, xmin:xmax]
    resized = cv.resize(crop, (0,0), fx=0.5, fy=0.5)
    
    meanR, varR, skewR = getColorMoment(crop[:,:,0]/255)
    meanG, varG, skewG = getColorMoment(crop[:,:,1]/255)
    meanB, varB, skewB = getColorMoment(crop[:,:,2]/255)
    
    meanR_list.append(meanR)
    varR_list.append(varR)
    skewR_list.append(skewR)
    meanG_list.append(meanG)
    varG_list.append(varG)
    skewG_list.append(skewG)
    meanB_list.append(meanB)
    varB_list.append(varB)
    skewB_list.append(skewB)

df['meanR'] = meanR_list
df['varR'] = varR_list
df['skewR'] = skewR_list
df['meanG'] = meanG_list
df['varG'] = varG_list
df['skewG'] = skewG_list
df['meanB'] = meanB_list
df['varB'] = varB_list
df['skewB'] = skewB_list

print(df)

       meanR      varR     skewR     meanG      varG     skewG     meanB  \
0   0.592339  0.257663 -0.132008  0.290304  0.182271  0.117737  0.158110   
1   0.792547  0.119288 -0.140294  0.740461  0.171134 -0.197484  0.553108   
2   0.737810  0.144685 -0.186657  0.569900  0.220292 -0.211651  0.360856   
3   0.753452  0.144378 -0.145630  0.565843  0.249226  0.156195  0.211582   
4   0.691034  0.126341 -0.025301  0.297600  0.111458  0.101749  0.028737   
5   0.596599  0.188079 -0.164606  0.458017  0.216170  0.071294  0.275069   
6   0.543856  0.147599  0.088442  0.299841  0.167531  0.185295  0.237076   
7   0.774546  0.099813 -0.124446  0.583060  0.125903 -0.126253  0.278734   
8   0.718737  0.137620 -0.123880  0.579314  0.231253 -0.129810  0.379069   
9   0.586451  0.277195 -0.176158  0.423994  0.263867  0.245839  0.291221   
10  0.711822  0.195686 -0.213459  0.358597  0.180763 -0.042263  0.097729   
11  0.740504  0.070958 -0.072460  0.371684  0.182929  0.172250  0.277640   
12  0.497998

In [9]:
from skimage.feature import greycomatrix, greycoprops
gray_list = []
for i in range(len(X)):
    gray = cv.cvtColor(X[i], cv.COLOR_RGB2GRAY)
    
    h = gray.shape[0]
    w = gray.shape[1]
    ymin, ymax, xmin, xmax = h//3, h*2//3, w//3, w*2//3
    crop = gray[ymin:ymax, xmin:xmax]
    
    resized = cv.resize(crop, (0,0), fx=0.5, fy=0.5)
    gray_list.append(crop)

In [10]:
def calc_glcm_all_agls(img, label, props, dists=[5], agls=[0, np.pi/4, np.pi/2, 3*np.pi/4], lvl=256, sym=True, norm=True):
    
    glcm = greycomatrix(img, 
                        distances=dists, 
                        angles=agls, 
                        levels=lvl,
                        symmetric=sym, 
                        normed=norm)
    feature = []
    glcm_props = [propery for name in props for propery in greycoprops(glcm, name)[0]]
    for item in glcm_props:
            feature.append(item)
    feature.append(label) 
    
    return feature


# ----------------- call calc_glcm_all_agls() for all properties ----------------------------------
properties = ['dissimilarity', 'correlation', 'homogeneity', 'contrast', 'ASM', 'energy']

glcm_all_agls = []
for img, label in zip(gray_list, labels): 
    glcm_all_agls.append(
            calc_glcm_all_agls(img, 
                                label, 
                                props=properties)
                            )
 
columns = []
angles = ['0', '45', '90','135']
for name in properties :
    for ang in angles:
        columns.append(name + "_" + ang)
        
columns.append("label")

glcm_df = pd.DataFrame(glcm_all_agls, 
                      columns = columns)

In [11]:
new_df = pd.concat([df,glcm_df], axis=1)
new_df

,meanR,varR,skewR,meanG,varG,skewG,meanB,varB,skewB,dissimilarity_0,...,contrast_135,ASM_0,ASM_45,ASM_90,ASM_135,energy_0,energy_45,energy_90,energy_135,label
0,0.592339,0.257663,-0.132008,0.290304,0.182271,0.117737,0.158110,0.066354,0.065945,11.037518,...,342.016637,0.000278,0.000228,0.000227,0.000233,0.016682,0.015107,0.015065,0.015274,FriedFood
1,0.792547,0.119288,-0.140294,0.740461,0.171134,-0.197484,0.553108,0.229585,-0.165997,19.667442,...,929.886945,0.000183,0.000158,0.000186,0.000181,0.013510,0.012564,0.013634,0.013452,FriedFood
2,0.737810,0.144685,-0.186657,0.569900,0.220292,-0.211651,0.360856,0.182091,0.138130,14.286867,...,934.963677,0.000423,0.000276,0.000267,0.000301,0.020558,0.016610,0.016341,0.017339,FriedFood
3,0.753452,0.144378,-0.145630,0.565843,0.249226,0.156195,0.211582,0.267733,0.269955,12.518143,...,612.008821,0.000223,0.000216,0.000207,0.000203,0.014923,0.014694,0.014396,0.014248,FriedFood
4,0.691034,0.126341,-0.025301,0.297600,0.111458,0.101749,0.028737,0.032224,0.039655,11.358522,...,294.608233,0.000330,0.000305,0.000360,0.000355,0.018162,0.017456,0.018976,0.018849,FriedFood
